# 1. 중급자를 위한 파이썬 

## 1.3. 데코레이터 (annotation) 上

* 가끔 파이썬 코드를 보면 ```@``` 기호가 붙은것을 많이 보았을 것입니다.
* 이것은 데코레이터라 하는것이며, 개인적으로 매직메서드와 더불어 파이썬 초급과 중급을 나누는 기준이라 생각합니다.
* 해당 기호는 무엇이고, 어디에 쓰이는지 한번 알아보도록 할꺄요?

### 1.3.1. 다중 함수 처리

* 우리가 코드를 짤 때, 하나의 결과물을 가지고 다른 함수를 실행해야 한다 가정하겠습니다.

In [6]:
def is_human_questionmark(living_being):
    ranks = ['iron', 'bronze', 'silver', 'gold', 'platinum', 'diamond']
    
    if not living_being in ranks:
        raise Exception('wrong tier')
    
    if living_being == 'iron':
        return False
    elif living_being == 'bronze':
        return False
    elif living_being == 'silver':
        return False
    else:
        return True

In [16]:
def join_game(**player):
    switch = is_human_questionmark(player['tier'])
    if switch:
        print(f"Welcome to the summoner's rift {player['name']}, {player['tier']} player!")
    else:
        print(f"You've banned from the game, sorry.")

In [151]:
player = {'name': 'gangsan', 'tier': 'diamond'}
join_game(**player)

Welcome to the summoner's rift gangsan, diamond player!


* 별 문제가 없는 코드입니다. 대부분이 이런식으로 하나의 함수 결과물을 판단하여 다른 함수의 행동을 결정지을때가 많습니다.
* 하지만 언제까지 초급 파이썬 사용자처럼 코드를 짤 수는 없습니다.
* 이것을 데코레이터를 활용해서 작성할 수 있지 않을까요?
* 데코레이터를 활용해서 작성해보기 전, 데코레이터가 어떻게 형성이 되는지 차근차근 뜯어봅시다.
* 만약 하나의 함수에 반복적인 행동을 하는 기능이 있다고 가정해봅시다.

In [23]:
def player_one():
    print('------player joined----')
    print('player_one')
    print('------player exited----')

def player_two():
    print('------player joined----')
    print('player_two')
    print('------player exited----')

def player_sam():
    print('------player joined----')
    print('player_sam')
    print('------player exited----')


In [24]:
player_one()

------player joined----
player_one
------player exited----


In [25]:
player_two()

------player joined----
player_two
------player exited----


In [26]:
player_sam()

------player joined----
player_sam
------player exited----


* ```plyer_``` 라는 함수가 호출될때마다 우리는 안내 문구를 띄우고 싶다 가정해봅시다.
* 함수마다 위처럼 print를 쓸 수도 있지만, 우리가 배우려는 데코레이터를 활용하면 저런짓이 필요 없겠다 가늠이 되겠죠?
* 데코레이터에 다이브 하기전 우리는 몇가지 CS 지식을 배우고 가야할 필요가 있습니다!

### 1.3.2. 일급 객체 (first class object)
* 데코레이터를 알기 전에 선행적으로 알아야 할것이 있습니다. 바로 일급 객체입니다.
* 일급 객체란 다른 객체들에 일반적으로 적용 가능한 연산을 모두 지원하는 객체를 말합니다.
* 일급 객체가 되기 위해선 아래 4가지 조건을 충족 해야 합니다:
    1. 변수에 담을 수 있다
    2. 파라메타로 전달 할 수 있다.
    3. 반환값으로 사용될 수 있다.
    4. 비교의 대상이 될 수 있다.

In [45]:
def square(x):
    return x * x

In [46]:
print(square(5)) # print() 함수의 파라메다로 전달 되었습니다.

25


In [51]:
fx = square      # 변수 fx 에 square 함수를 할당하였습니다.

In [48]:
print(square)

<function square at 0x7f14a40865e0>


In [49]:
print(fx)

<function square at 0x7f14a40865e0>


In [50]:
print(fx(5))     # fx()를 실행시켜 square함수를 실행, 즉 반환값으로 사용되었습니다.

25


* 위에서 본것처럼 함수 ```fx()``` 함수를 호출하여 ```square``` 함수를 호출하였습니다.
* 더 실용성있게 구성해볼까요?

In [60]:
def square(x):
    return x * x

def mapper(f, *args):
    result = []
    for i in args:
        result.append(f(i))
    return result

In [61]:
num_list = 1,2,3,4,5,6
result = mapper(square, *num_list)
print(result)

[1, 4, 9, 16, 25, 36]


* 이런식으로 함수를 일급 객체 취급하여 얻는 이점은, 우리가 작성한 함수를 변형, 응용할 수 있다는 점입니다.
* 함수가 일급 객체라는것은 함수를 객체와 동일하게 사용할 수 있다는 의미입니다!

In [57]:
def minus_one(x):
    return x - 1

def double_it(x):
    return x * 2

In [59]:
result2 = mapper(minus_one, *num_list)
print(result2)

[0, 1, 2, 3, 4, 5]


In [62]:
result3 = mapper(double_it, *num_list)
print(result3)

[2, 4, 6, 8, 10, 12]


* 신기하죠? 이게 일급 객체입니다.
* 여기서 일급객체함수 (```square```, ```minus_one```, ```double_it```)를 받아 처리하는 함수 (```mapper```) 를 고차함수
* (higher-order function) 이라 합니다.
* 여기서 우리가 다룬 일급 객체 함수를 또 다른 말로 콜백함수 (callback function) 라고도 합니다.

### 1.3.3. 클로져 (closure)
* 클로져는 일급객체함수를 지원하는 언어가 지원하는 기술입니다. 그렇기 때문에 클로져를 배우기 전 일급 객체함수를 알아야 합니다.
* 우리의 목적 데코레이터를 알기 전에 또한 이 클로져의 개념이 필요합니다.
* 아래의 예제를 보고 클로져를 이해해보도록 합시다.

In [77]:
def pretty_print():
    msg = '------player joined----'
    msg2 = '------player exited----'
    
    def player_five():
        print(msg)
        print('5p')
        print(msg2)
    
    return player_five()

In [80]:
pretty_print()

------player joined----
5p
------player exited----


* 어 이상합니다. 분명히 ```msg```, ```msg2``` 는 ```pretty_print()```의 지역변수 이고,
* 해당 인자들은 ```player_five()``` 함수로 전달이 되지 않는데 어떻게 출력이 되었을까요??
* 그걸 알기 위해서 함수를 살짝 변형해보겠습니다.

In [82]:
def pretty_print2():
    msg = '------player joined----'
    msg2 = '------player exited----'
    
    def player_five():
        print(msg)
        print('5p')
        print(msg2)
    
    return player_five # <-------------- () 만 지웠습니다!

In [84]:
pretty_print2()

<function __main__.pretty_print2.<locals>.player_five()>

* 위의 함수는 return 시 ```player_five``` 함수를 호출하지 않고 그 함수 오브젝트를 리턴하고 있습니다.
* 또한 이 함수 안에는 ```player_five``` 가 할당되어 있는게 보입니다.

In [88]:
tag = pretty_print2()

In [91]:
print(tag)

<function pretty_print2.<locals>.player_five at 0x7f14a40b38b0>


In [94]:
tag()

------player joined----
5p
------player exited----


In [95]:
print(tag.__dir__())

['__repr__', '__call__', '__get__', '__new__', '__closure__', '__doc__', '__globals__', '__module__', '__code__', '__defaults__', '__kwdefaults__', '__annotations__', '__dict__', '__name__', '__qualname__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__init__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


In [122]:
print(tag.__class__, tag.__name__) # tag 는 결국 player_five 함수 자체라는것을 알 수 있습니다.

<class 'function'> player_five


In [100]:
print(tag.__closure__)

(<cell at 0x7f148fd15be0: str object at 0x7f14a409c9e0>, <cell at 0x7f148ffc5460: str object at 0x7f148feb3a80>)


In [101]:
print(tag.__closure__.__dir__())

['__repr__', '__hash__', '__getattribute__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__iter__', '__len__', '__getitem__', '__add__', '__mul__', '__rmul__', '__contains__', '__new__', '__getnewargs__', 'index', 'count', '__doc__', '__str__', '__setattr__', '__delattr__', '__init__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


In [105]:
print(type(tag.__closure__))

<class 'tuple'>


In [103]:
print(tag.__closure__[0])

<cell at 0x7f148fd15be0: str object at 0x7f14a409c9e0>


In [107]:
print(type(tag.__closure__[0]))

<class 'cell'>


In [109]:
print(tag.__closure__[0].__dir__())

['__repr__', '__getattribute__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__new__', 'cell_contents', '__doc__', '__hash__', '__str__', '__setattr__', '__delattr__', '__init__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


In [110]:
print(tag.__closure__[0].cell_contents)

------player joined----


In [111]:
print(tag.__closure__[1].cell_contents)

------player exited----


* 아하.. ```cell_contents``` 라는곳에 값이 저장이 되어 있습니다.
* 클로저는 어떤 함수를 함수 자신이 가지고 있는 환경과 함께 저장한 레코드입니다.
* 함수가 가진 프리변수 (free variable)를 클로져가 만들어지는 당시의 값과 레퍼런스 (cell_contents)에 저장합니다.
* 즉 클로져는 일반 함수와는 다르게, 자신의 영역 밖에서 호출된 함수의 변수값과 레퍼런스를 복사, 저장 후 이를 엑세스할 수 있게 해줍니다.
* 클로져를 사용하면 아래처럼 함수를 가지고 놀 수 있습니다:

In [124]:
def pretty_print3(dec):
    decoration = dec
    
    def shazam(text):
        doc = text
        print(f'{dec} {doc} {dec}')
    
    return shazam

In [125]:
cool_deco = pretty_print3('~*~*~*~')
nice_deco = pretty_print3('!!!___!!!')
amazing_deco = pretty_print3('^~~~^')

In [150]:
text = 'damn this text is fancy'
cool_deco(text)
print() # 걍 이쁘게 보이기 위해서 라인 추가
nice_deco(text)
print()
amazing_deco(text)

~*~*~*~ damn this text is fancy ~*~*~*~

!!!___!!! damn this text is fancy !!!___!!!

^~~~^ damn this text is fancy ^~~~^


* 자바스크립트에서 말하는 일급 객체, 클로져가 이런것입니다.
* 이렇게 파이썬도 자바스크립트처럼 함수형 프로그래밍을 지원합니다!

### 1.3.4. 데코레이터 (@)
* 좀 많이 돌아온 감이 없잖아 있지만, 이제 진짜 데코레이터가 어떻게 동작되는지 한번 살펴보겠습니다.

In [140]:
def printing(f):
    def join_and_exit():
        print('------player joined----')
        f()
        print('------player exited----')
    return join_and_exit

In [142]:
def player_sa():
    print('4p')

In [143]:
wow = printing(player_sa)

In [145]:
wow()

------player joined----
4p
------player exited----


* 우리는 이제 이 함수가 어떻게 작성이 되어 있는지 보입니다.
* ```player_sa``` 라는 함수가 일급객체 취급이 되어 ```printing``` 이라는 고차함수의 인자로 들어갔으며,
* 고차함수 ```printing``` 안의 함수 ```join_and_exit``` 함수가 아무 인자를 받지 않는데도 불구하고
* 상위 함수로부터 받은 인자 ```f```를 받아서 처리 하고 있습니다.
* 실로 일급객체와 클로져가 잘 버무려진 함수가 아닐 수 없습니다.
* 이걸 ```@``` 형식으로 적용하면 어떻게 될까요?

In [146]:
@printing
def player_six():
    print('6p')

In [147]:
player_six()

------player joined----
6p
------player exited----


* 아 별거 없습니다, 우리가 만든 고차함수 printing을 그냥 ```@printing```으로 적어버리면 되는겁니다.
* 위의 코드와 비교하면, 고차함수 ```printing```에 ```player_sa```함수를 인자로 넘긴것을 ```wow```라는 변수로 받고,
* 그 ```wow```를 실행시킨것을 ```@printing``` 하나로 처리 해버린것입니다.
* 이제 데코레이터를 배우기 위해 왜 일급객체와 클로져가 필요한지 이해가 가셨나요?
* 다음시간엔 더 깊숙히 데코레이터의 심연으로 다이브 해 봅시다.